In [17]:
import os
import cv2
import numpy as np
from collections import defaultdict
import random


image_dir = r"C:\Users\Projects\Age_Gender_classifier\images"
img_size = 64

# Clear old data
X = []
age_labels = []
gender_labels = []
race_labels = []

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        parts = filename.split("_")
        if len(parts) >= 4:
            try:
                age = int(parts[0])
                gender = int(parts[1])
                race = int(parts[2])

                img_path = os.path.join(image_dir, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))
                img = img.astype("float32") / 255.0

                X.append(img)
                age_labels.append(age)
                gender_labels.append([1, 0] if gender == 0 else [0, 1])
                race_labels.append(race)

            except Exception as e:
                print(f"❌ Skipping {filename}: {e}")
        else:
            print(f"⚠️ Skipping malformed filename: {filename}")

# Convert to arrays
X = np.array(X)
age_labels = np.array(age_labels)
gender_labels = np.array(gender_labels)
race_labels = np.array(race_labels)

# Final check
print("✅ Loaded:")
print("X shape         :", X.shape)
print("Age labels      :", age_labels.shape)
print("Gender labels   :", gender_labels.shape)
print("Race labels     :", race_labels.shape)


⚠️ Skipping malformed filename: 39_1_20170116174525125.jpg
❌ Skipping 53__0_20170116184028385.jpg: invalid literal for int() with base 10: ''
⚠️ Skipping malformed filename: 61_1_20170109142408075.jpg
⚠️ Skipping malformed filename: 61_3_20170109150557335.jpg
✅ Loaded:
X shape         : (24102, 64, 64, 3)
Age labels      : (24102,)
Gender labels   : (24102, 2)
Race labels     : (24102,)


In [52]:

assert len(age_labels) == len(gender_labels) == len(race_labels) == len(X)

groups = defaultdict(list)
for idx, (age, g_onehot, race) in enumerate(zip(age_labels, gender_labels, race_labels)):
    gender_int = int(np.argmax(g_onehot))  # 0 or 1
    key = (age, gender_int, race)
    groups[key].append(idx)

random.seed(42)
train_idx, test_idx = [], []

for idx_list in groups.values():
    random.shuffle(idx_list)
    n = len(idx_list)

    if n == 1:
        train_idx += idx_list
    elif n == 2:
        train_idx += [idx_list[0]]
        test_idx  += [idx_list[1]]
    else:
        k = int(round(0.8 * n))
        train_idx += idx_list[:k]
        test_idx  += idx_list[k:]

print(f"Train: {len(train_idx)}  Test: {len(test_idx)}  Total: {len(train_idx)+len(test_idx)}")

# --- slice images & labels ---
X_train       = X[train_idx]
age_train     = age_labels[train_idx]
gender_train  = gender_labels[train_idx]      # ← add this

X_test        = X[test_idx]
age_test      = age_labels[test_idx]
gender_test   = gender_labels[test_idx]       # ← add this


Train: 19241  Test: 4861  Total: 24102


In [40]:
age_train = age_labels[train_idx].reshape(-1, 1)   #  (19241, 1)
age_test  = age_labels[test_idx].reshape(-1, 1)    #  ( 4861, 1)



In [41]:
# --- 1. Reshape the images to flat vectors (for feedforward NN)
X_train_flat = X_train.reshape(X_train.shape[0], -1).astype("float32")  # shape: (19241, 12288)
X_test_flat  = X_test.reshape(X_test.shape[0], -1).astype("float32")    # shape: (4861, 12288)

# --- 2. Scale the age values to [0, 1]
MAX_AGE = age_train.max()   # e.g. 116
age_train_scaled = age_train / MAX_AGE
age_test_scaled  = age_test  / MAX_AGE




In [25]:
# ----------------------------------------------------------------------
# Helpers
# ----------------------------------------------------------------------
def he_init(fan_in, fan_out):
    return np.random.randn(fan_in, fan_out) * np.sqrt(2.0 / fan_in)

def relu(x):              return np.maximum(0., x)
def relu_grad(x):         return (x > 0.).astype(np.float32)

def softmax(z):
    exp = np.exp(z - z.max(axis=1, keepdims=True))
    return exp / exp.sum(axis=1, keepdims=True)

def mse(pred, target):
    return np.mean((pred - target) ** 2)

def mse_grad(pred, target):
    return 2 * (pred - target) / target.shape[0]

def cross_entropy(pred, target, eps=1e-12):
    pred = np.clip(pred, eps, 1 - eps)
    return -np.mean(np.sum(target * np.log(pred), axis=1))

def cross_entropy_grad(pred, target):
    return (pred - target) / target.shape[0]


In [47]:
class Dense:
    def __init__(self, in_dim, out_dim):
        self.W = np.random.randn(in_dim, out_dim) * np.sqrt(2.0 / in_dim)
        self.b = np.zeros((1, out_dim))
        self.X = None                      # cache for backward
        self.dW, self.db = None, None

    def forward(self, X):
        self.X = X
        return X @ self.W + self.b

    def backward(self, dY):
        """
        dY: gradient from next layer, shape (batch, out_dim)
        Returns dX: gradient wrt input, shape (batch, in_dim)
        NOTE: gradients are averaged over the batch (safer updates)
        """
        m  = dY.shape[0]                          # batch size
        self.dW = (self.X.T @ dY) / m
        self.db = np.mean(dY, axis=0, keepdims=True)
        dX = dY @ self.W.T
        return dX

    def step(self, lr):
        self.W -= lr * self.dW
        self.b -= lr * self.db


In [48]:
# ----------------------------------------------------------------------
# Multi‑Task Network
# ----------------------------------------------------------------------
class MultiTaskNN:
    """
    Parameters
    ----------
    input_dim : int   (12288 for 64x64x3)
    alpha     : float weight on age   loss
    beta      : float weight on gender loss
    """

    def __init__(self, input_dim, alpha=1.0, beta=1.0):
        self.alpha, self.beta = alpha, beta

        # Shared trunk
        self.s1 = Dense(input_dim, 512)
        self.s2 = Dense(512, 256)

        # Age head
        self.a1 = Dense(256, 64)
        self.a2 = Dense(64, 1)

        # Gender head
        self.g1 = Dense(256, 64)
        self.g2 = Dense(64, 2)

    # -------------------- forward --------------------
    def forward(self, X):
        z1 = self.s1.forward(X); a1 = relu(z1)
        z2 = self.s2.forward(a1); a2 = relu(z2)

        # Age
        za1 = self.a1.forward(a2); aa1 = relu(za1)
        age_pred = self.a2.forward(aa1)           # (B,1)

        # Gender
        zg1 = self.g1.forward(a2); ag1 = relu(zg1)
        logits = self.g2.forward(ag1)
        gender_pred = softmax(logits)             # (B,2)

        cache = (z1, a1, z2, a2, za1, aa1, zg1, ag1, logits)
        return age_pred, gender_pred, cache

    # -------------------- backward -------------------
    def backward(self, cache, age_pred, gender_pred,
                 age_true, gender_true, lr):
        z1,a1,z2,a2,za1,aa1,zg1,ag1,logits = cache

        # --- gradients w.r.t losses
        d_age   = self.alpha * mse_grad(age_pred, age_true)        # (B,1)
        d_gen   = self.beta  * cross_entropy_grad(gender_pred,
                                                  gender_true)     # (B,2)

        # --- Age head backprop
        d = self.a2.backward(d_age);        # (B,64)
        d = relu_grad(za1) * d
        d = self.a1.backward(d)             # (B,256)

        # --- Gender head backprop
        dg = self.g2.backward(d_gen);       # (B,64)
        dg = relu_grad(zg1) * dg
        dg = self.g1.backward(dg)           # (B,256)

        # Combine gradients from both heads
        d_shared = d + dg

        # --- Shared trunk backprop
        d_shared = self.s2.backward(relu_grad(z2) * d_shared)
        _        = self.s1.backward(relu_grad(z1) * d_shared)

        # --- Update all layers
        for layer in (self.s1, self.s2,
                      self.a1, self.a2, self.g1, self.g2):
            layer.step(lr)

    # -------------------- training step --------------
    def train_batch(self, X, age_true, gender_true, lr=1e-3):
        age_pred, gender_pred, cache = self.forward(X)
        loss_age  = mse(age_pred, age_true)
        loss_gen  = cross_entropy(gender_pred, gender_true)
        loss_tot  = self.alpha * loss_age + self.beta * loss_gen

        self.backward(cache, age_pred, gender_pred,
                      age_true, gender_true, lr)

        return loss_tot, loss_age, loss_gen

    # -------------------- inference ------------------
    def predict(self, X):
        age_pred, gender_pred, _ = self.forward(X)
        return age_pred, gender_pred

In [59]:
# --- 3. Define and train the model
model = MultiTaskNN(input_dim=X_train_flat.shape[1], alpha=1.0, beta=5.0)

epochs = 25
batch  = 128
lr     = 1e-4

num_train = X_train_flat.shape[0]

for epoch in range(1, epochs + 1):
    # ---- Shuffle all three training arrays the same way
    perm = np.random.permutation(num_train)
    X_train_flat     = X_train_flat[perm]
    age_train_scaled = age_train_scaled[perm]
    gender_train     = gender_train[perm]

    tot, n = 0.0, 0
    for i in range(0, num_train, batch):
        xb = X_train_flat[i : i + batch]
        ab = age_train_scaled[i : i + batch]
        gb = gender_train[i : i + batch]

        loss_tot, _, _ = model.train_batch(xb, ab, gb, lr)
        tot += loss_tot * xb.shape[0]
        n   += xb.shape[0]

    print(f"Epoch {epoch:02d} | Loss {tot / n:.4f}")

Epoch 01 | Loss 4.5391
Epoch 02 | Loss 3.9458
Epoch 03 | Loss 3.7603
Epoch 04 | Loss 3.6948
Epoch 05 | Loss 3.6683
Epoch 06 | Loss 3.6543
Epoch 07 | Loss 3.6458
Epoch 08 | Loss 3.6395
Epoch 09 | Loss 3.6338
Epoch 10 | Loss 3.6287
Epoch 11 | Loss 3.6241
Epoch 12 | Loss 3.6196
Epoch 13 | Loss 3.6154
Epoch 14 | Loss 3.6113
Epoch 15 | Loss 3.6072
Epoch 16 | Loss 3.6034
Epoch 17 | Loss 3.5997
Epoch 18 | Loss 3.5961
Epoch 19 | Loss 3.5926
Epoch 20 | Loss 3.5893
Epoch 21 | Loss 3.5859
Epoch 22 | Loss 3.5828
Epoch 23 | Loss 3.5796
Epoch 24 | Loss 3.5765
Epoch 25 | Loss 3.5735


In [50]:
def train_model(model, X_train_flat, age_train_scaled, gender_train,
                X_val_flat=None, age_val_scaled=None, gender_val=None,
                max_age=100, epochs=25, batch=128, lr=1e-4):
    """
    Trains the model and returns histories.
    """
    num_train = X_train_flat.shape[0]
    history_loss, history_mae, history_acc = [], [], []

    for epoch in range(1, epochs + 1):
        # ---- shuffle every epoch
        perm = np.random.permutation(num_train)
        X_train_flat     = X_train_flat[perm]
        age_train_scaled = age_train_scaled[perm]
        gender_train     = gender_train[perm]

        tot_loss, seen = 0.0, 0
        for i in range(0, num_train, batch):
            xb = X_train_flat[i:i+batch]
            ab = age_train_scaled[i:i+batch]
            gb = gender_train[i:i+batch]

            loss_tot, _, _ = model.train_batch(xb, ab, gb, lr)
            tot_loss += loss_tot * xb.shape[0]
            seen     += xb.shape[0]

        # ---- end‑of‑epoch metrics on full train set
        age_pred_sc, g_pred = model.predict(X_train_flat)
        age_pred = age_pred_sc.squeeze() * max_age
        mae_epoch = np.mean(np.abs(age_pred - (age_train_scaled.squeeze() * max_age)))
        acc_epoch = np.mean(np.argmax(g_pred,1) == np.argmax(gender_train,1))

        history_loss.append(tot_loss / seen)
        history_mae.append(mae_epoch)
        history_acc.append(acc_epoch)

        print(f"Epoch {epoch:02d} | Loss {tot_loss/seen:.4f} | "
              f"MAE {mae_epoch:.2f} | Acc {acc_epoch*100:.2f}%")

    return history_loss, history_mae, history_acc


def evaluate_model(model, X_flat, age_true, gender_true, max_age=100):
    """
    Returns MAE for age and accuracy for gender on a dataset.
    """
    age_pred_sc, g_pred = model.predict(X_flat)
    age_pred = age_pred_sc.squeeze() * max_age

    from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix
    mae = mean_absolute_error(age_true.squeeze(), age_pred)
    acc = accuracy_score(np.argmax(gender_true,1), np.argmax(g_pred,1))
    cm  = confusion_matrix(np.argmax(gender_true,1), np.argmax(g_pred,1))
    return mae, acc, cm


In [ ]:
# --- assume you already prepared:
# X_train_flat, age_train_scaled, gender_train
# X_test_flat,  age_test,         gender_test
# model = MultiTaskNN(...)

hist_loss, hist_mae, hist_acc = train_model(
    model,
    X_train_flat, age_train_scaled, gender_train,
    max_age=MAX_AGE, epochs=25, batch=128, lr=1e-4
)

mae, acc, cm = evaluate_model(
    model, X_test_flat, age_test, gender_test, max_age=MAX_AGE
)
print(f"\n📊 Test MAE: {mae:.2f} years | Gender Accuracy: {acc*100:.2f}%")
print("Confusion matrix:\n", cm)


In [57]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history_loss); plt.title("Total loss"); plt.xlabel("epoch"); plt.ylabel("loss"); plt.show()
plt.figure()
plt.plot(history_mae);  plt.title("Age MAE");    plt.xlabel("epoch"); plt.ylabel("years"); plt.show()
plt.figure()
plt.plot(history_acc);  plt.title("Gender accuracy"); plt.xlabel("epoch"); plt.ylabel("acc"); plt.show()


NameError: name 'history_loss' is not defined

<Figure size 640x480 with 0 Axes>